In [1]:
import tensorflow as tf
import numpy as np

/home/lucio/anaconda3/envs/Python3.6_latest_versions/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [8]:
# Create a tensor of shape [2, 3] consisting of random normal values, with mean
# -1 and standard deviation 4.
norm = tf.random_normal([2, 3], mean=-1, stddev=4)


# Each time we run these ops, different results are generated
sess = tf.Session()
print(sess.run(norm))


[[ 1.14153576 -1.47401798 -3.46384096]
 [-1.49624348 -5.0458889  -2.27493286]]


In [41]:
# Initial Conditions -- some rain drops hit a pond
N = 1000
# Set everything to zero
u_init = np.zeros([N], dtype=np.float32)
ut_init = np.zeros([N], dtype=np.float32)



In [42]:
# Parameters:
sigma = tf.placeholder(tf.float32, shape=())

# Create variables for simulation state
r  = tf.Variable(u_init)
delta_r = tf.Variable(ut_init)

# Discretized PDE update rules
r_ = r + delta_r
delta_r_ = tf.random_normal([N], mean=1, stddev=4)*sigma

# Operation to update the state
step = tf.group(
  r.assign(r_),
  delta_r.assign(delta_r_))

In [46]:
%%time
sess = tf.InteractiveSession()
# Initialize state to initial conditions
tf.global_variables_initializer().run()

# Run 1000 steps of PDE

for i in range(40000):
# Step simulation
    step.run({sigma: 0.03})
print(r.eval().shape)

(1000,)
CPU times: user 10.1 s, sys: 584 ms, total: 10.7 s
Wall time: 6.64 s


In [48]:
%%time
r = np.zeros(1000)
for i in range(40000):
# Step simulation
    r = r + np.random.normal(1000)
print(r.shape)

(1000,)
CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 133 ms


# Internal loop

In [35]:
jit_scope = tf.contrib.compiler.jit.experimental_jit_scope

i = tf.constant(0)
k = tf.constant(np.zeros((1000)))
i_k = (i,k)

with jit_scope():
    c = lambda i, k: i<40000
    norm = tf.random_normal([1000], mean=0, stddev=2, dtype=tf.float64)
    b = lambda i,k:(i+1, k+norm**2)
_,r = tf.while_loop(c, b, i_k)

In [37]:
%%time
# Config to turn on JIT compilation
config = tf.ConfigProto()
config.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1

sess = tf.Session(config=config)
#sess = tf.Session()
# Initialize state to initial conditions
out = sess.run(r)
#     tf.global_variables_initializer().run()
     
print(out.shape)

(1000,)
CPU times: user 1.09 s, sys: 164 ms, total: 1.26 s
Wall time: 398 ms
